In [19]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('../data_cache/AaveV2LiquidityPoolLoader')

In [5]:
df.head()

,tx_hash,block_number,tx_timestamp,log_index,log_ordinal,action_name,reserve_addr,user_addr,on_behalf_of_addr,repayer_addr,withdraw_to_addr,amount,borrow_rate_mode,borrow_rate,referral
0,7f10004def891a9965e972e57ecb8b93c517b5d4d7e453...,11367463,2020-12-01 15:29:15+00,8,4151,Deposit,7fc66500c84a76ad7e9c93437bfc5ac33e2ddae9,dad4c11e8cc6a5c37808d3b31b3b284809f702d1,dad4c11e8cc6a5c37808d3b31b3b284809f702d1,NaN,NaN,100000000000000000,NaN,NaN,0.0
1,dd1d259637c499ff61b0d142e7bb6639f3552b2dab374c...,11367499,2020-12-01 15:35:49+00,6,3245,Deposit,c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,dcd33426ba191383f1c9b431a342498fdac73488,dad4c11e8cc6a5c37808d3b31b3b284809f702d1,NaN,NaN,1000000000000000,NaN,NaN,0.0
2,0c66ade8a26d0bedbc967005a75a00028cebf222ec45e2...,11367541,2020-12-01 15:42:19+00,10,3422,Withdraw,c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,dcd33426ba191383f1c9b431a342498fdac73488,NaN,NaN,dcd33426ba191383f1c9b431a342498fdac73488,1000000000000000,NaN,NaN,NaN
3,870bf17a6373c10290d626134936230ada5f8ab0eee2f9...,11367752,2020-12-01 16:30:48+00,6,6054,Deposit,c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,dcd33426ba191383f1c9b431a342498fdac73488,bd723fc4f1d737dcfc48a07fe7336766d34cad5f,NaN,NaN,1000000000000000,NaN,NaN,0.0
4,25f636e7b49a599c765bcdac00853265ac76a592195346...,11367770,2020-12-01 16:35:29+00,10,5562,Withdraw,c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,dcd33426ba191383f1c9b431a342498fdac73488,NaN,NaN,dcd33426ba191383f1c9b431a342498fdac73488,1000000000000000,NaN,NaN,NaN


In [21]:
usdt_deposit_df = df[(df.reserve_addr=='dac17f958d2ee523a2206206994597c13d831ec7') & (df.action_name == 'Deposit')]
usdt_withdraw_df = df[(df.reserve_addr=='dac17f958d2ee523a2206206994597c13d831ec7') & (df.action_name == 'Withdraw')]
usdt_borrow_df = df[(df.reserve_addr=='dac17f958d2ee523a2206206994597c13d831ec7') & (df.action_name == 'Borrow')]
usdt_repay_df = df[(df.reserve_addr=='dac17f958d2ee523a2206206994597c13d831ec7') & (df.action_name == 'Repay')]
eth_deposit_df = df[(df.reserve_addr=='c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2') & (df.action_name == 'Deposit')]
eth_withdraw_df = df[(df.reserve_addr=='c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2') & (df.action_name == 'Withdraw')]
eth_borrow_df = df[(df.reserve_addr=='c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2') & (df.action_name == 'Borrow')]
eth_repay_df = df[(df.reserve_addr=='c02aaa39b223fe8d0a0e5c4f27ead9083c756cc2') & (df.action_name == 'Repay')]

In [22]:
usdt_deposit_df.index = pd.to_datetime(usdt_deposit_df.tx_timestamp)
usdt_withdraw_df.index = pd.to_datetime(usdt_withdraw_df.tx_timestamp)
usdt_borrow_df.index = pd.to_datetime(usdt_borrow_df.tx_timestamp)
usdt_repay_df.index = pd.to_datetime(usdt_repay_df.tx_timestamp)
eth_deposit_df.index = pd.to_datetime(eth_deposit_df.tx_timestamp)
eth_withdraw_df.index = pd.to_datetime(eth_withdraw_df.tx_timestamp)
eth_borrow_df.index = pd.to_datetime(eth_borrow_df.tx_timestamp)
eth_repay_df.index = pd.to_datetime(eth_repay_df.tx_timestamp)

In [31]:
grouped_df = pd.concat([
    usdt_deposit_df['amount'].astype(float).resample('10min').agg(np.sum), 
    usdt_withdraw_df['amount'].astype(float).resample('10min').agg(np.sum), 
    usdt_borrow_df['amount'].astype(float).resample('10min').agg(np.sum), 
    usdt_repay_df['amount'].astype(float).resample('10min').agg(np.sum), 
    eth_deposit_df['amount'].astype(float).resample('10min').agg(np.sum), 
    eth_withdraw_df['amount'].astype(float).resample('10min').agg(np.sum), 
    eth_borrow_df['amount'].astype(float).resample('10min').agg(np.sum), 
    eth_repay_df['amount'].astype(float).resample('10min').agg(np.sum)], axis =1)
grouped_df.columns = ['usdt_deposit','usdt_withdraw','usdt_borrow','usdt_repay', 'eth_deposit','eth_withdraw', 'eth_borrow', 'eth_repay']
grouped_df = grouped_df.fillna(0)

In [34]:
grouped_df.to_csv('../processed_data/aave_10mins_activities.csv')